# Criação das Tabelas para o Dash de Clusters

### Importando as bibliotecas & Carregando os dados
---

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, row_number, lit, countDistinct, max, when, date_format, dayofweek, month, dayofmonth, hour, concat_ws, round
from pyspark.sql.window import Window

In [0]:
tabela_vendas_bronze = "estudo.default.tvendas_bronze"
tabela_prod_dados_cluster_cli = "estudo.default.tcluster_cli_gold"
tabela_dash_metricas_geral = "estudo.default.tcluster_dash_geral_gold"
tabela_temporal_ticket_dia_semana = "estudo.default.tcluster_dash_temporal_ticket_dia_semana_gold"
tabela_temporal_compras_mes = "estudo.default.tcluster_dash_temporal_compras_mes_gold"
tabela_temporal_compras_hora = "estudo.default.tcluster_dash_temporal_compras_hora_gold"
tabela_temporal_compras_estacao = "estudo.default.tcluster_dash_temporal_compras_estacao_gold"
tabela_temporal_compras_dia = "estudo.default.tcluster_dash_temporal_compras_dia_gold"
tabela_temporal_compras_rota = "estudo.default.tcluster_dash_temporal_compras_rota_gold"

In [0]:
df_prod_dados_cluster_cli = spark.table(tabela_prod_dados_cluster_cli)
df_vendas_bronze = spark.table(tabela_vendas_bronze)

### Variáveis gerais
---

In [0]:
df_agregado_numerico = df_prod_dados_cluster_cli.groupBy("ds_cluster", "dt_particao").agg(
    count(col("fk_contact")).alias("qt_total_cliente"),
    sum(col("vl_total")).alias("vl_total_compra"),
    sum(col("qt_compra")).alias("qt_total_compra"),
    sum(col("qt_total_ticket")).alias("qt_total_ticket_compra"),
    sum(col("qt_total_compra_volta")).alias("qt_total_compra_com_volta")
)

In [0]:
df_agregado_numerico = df_agregado_numerico.withColumn("qt_media_ticket_compra", round(col("qt_total_ticket_compra") / col("qt_total_compra"), 2)) \
    .withColumn("vl_medio_compra", round(col("vl_total_compra") / col("qt_total_compra"), 2)) \
    .withColumn("pc_compra_volta", round(col("qt_total_compra_com_volta") / col("qt_total_compra") * 100, 2))

In [0]:
def calculando_variaveis_preferidas_dash(df, group_col, mode_col_name):
    """
    Calcula a moda (valor mais frequente) para uma coluna dentro de um grupo.
    Retorna um DataFrame com a coluna de grupo e a moda.
    """
    df_dia_semana = df.groupBy(group_col, mode_col_name).agg(
    sum(col("qt_compra")).alias("qt")
    )

    window_spec = Window.partitionBy(group_col).orderBy(col("qt").desc())

    moda_df = (
            df_dia_semana.withColumn("rn", row_number().over(window_spec))
            .filter(col("rn") == 1)
            .select(group_col, mode_col_name)
        )
    return moda_df

In [0]:
df_dia_semana_preferido = calculando_variaveis_preferidas_dash(df_prod_dados_cluster_cli, "ds_cluster", "ds_dia_semana_preferido")
df_mes_preferido = calculando_variaveis_preferidas_dash(df_prod_dados_cluster_cli, "ds_cluster", "ds_mes_preferido")
df_periodo_dia_preferido = calculando_variaveis_preferidas_dash(df_prod_dados_cluster_cli, "ds_cluster", "nu_hora_preferido")

In [0]:
df_join_metricas_gerais = df_agregado_numerico \
    .join(df_dia_semana_preferido, on=["ds_cluster"], how="left") \
    .join(df_mes_preferido, on=["ds_cluster"], how="left") \
    .join(df_periodo_dia_preferido, on=["ds_cluster"], how="left")

In [0]:
df_join_metricas_gerais = (df_join_metricas_gerais
                          .withColumn("ds_periodo_dia_preferido", when((col("nu_hora_preferido") >= 0) & (col("nu_hora_preferido") <= 5), "Madrugada")
                                      .when((col("nu_hora_preferido") >= 6) & (col("nu_hora_preferido") <= 11), "Manhã")
                                      .when((col("nu_hora_preferido") >= 12) & (col("nu_hora_preferido") <= 17), "Tarde")
                                      .otherwise("Noite").alias("ds_periodo_dia_preferido")).drop("nu_hora_preferido"))

#### Escrita na tabela relacional geral
---

In [0]:
df_join_metricas_gerais.write.mode("overwrite").saveAsTable(tabela_dash_metricas_geral)

### Variáveis temporais
---

In [0]:
df_join = df_vendas_bronze.alias("bronze") \
    .join(df_prod_dados_cluster_cli.alias("prod"), on=["fk_contact"], how="left").select("bronze.*", "prod.ds_cluster")

In [0]:
df_enriquecido = df_join.withColumn("ano_mes_compra", date_format(col("date_purchase"), "yyyy-MM")) \
    .withColumn("ano_compra", date_format(col("date_purchase"), "yyyy")) \
    .withColumn("ds_mes_compra", month(col("date_purchase"))) \
    .withColumn("nu_dia_compra", dayofmonth(col("date_purchase"))) \
    .withColumn("ds_dia_semana_compra", dayofweek(col("date_purchase"))) \
    .withColumn("nu_hora_compra", hour(col("time_purchase"))) \
    .withColumn(
    "ds_estacao_do_ano",
        when((month(col("date_purchase")) == 12) & (dayofmonth(col("date_purchase")) >= 21) | \
            (month(col("date_purchase")) == 1) | \
            (month(col("date_purchase")) == 2) | \
            (month(col("date_purchase")) == 3) & (dayofmonth(col("date_purchase")) < 20), "Verão")
        .when((month(col("date_purchase")) == 3) & (dayofmonth(col("date_purchase")) >= 20) | \
            (month(col("date_purchase")) == 4) | \
            (month(col("date_purchase")) == 5) | \
            (month(col("date_purchase")) == 6) & (dayofmonth(col("date_purchase")) < 20), "Outono")
        .when((month(col("date_purchase")) == 6) & (dayofmonth(col("date_purchase")) >= 20) | \
            (month(col("date_purchase")) == 7) | \
            (month(col("date_purchase")) == 8) | \
            (month(col("date_purchase")) == 9) & (dayofmonth(col("date_purchase")) < 22), "Inverno")
        .when((month(col("date_purchase")) == 9) & (dayofmonth(col("date_purchase")) >= 22) | \
            (month(col("date_purchase")) == 10) | \
            (month(col("date_purchase")) == 11) | \
            (month(col("date_purchase")) == 12) & (dayofmonth(col("date_purchase")) < 21), "Primavera")
        .otherwise("Não Definido")
    ) \
    .withColumn("ds_rota_ida", concat_ws(" -> ", col("place_origin_departure"), col("place_destination_departure")))

In [0]:
df_tratado = df_enriquecido.withColumn("ds_dia_semana_compra", when(col("ds_dia_semana_compra") == 1, 'Dom')
                .when(col("ds_dia_semana_compra") == 2, 'Seg')
                .when(col("ds_dia_semana_compra") == 3, 'Ter')
                .when(col("ds_dia_semana_compra") == 4, 'Qua')
                .when(col("ds_dia_semana_compra") == 5, 'Qui')
                .when(col("ds_dia_semana_compra") == 6, 'Sex')
                .when(col("ds_dia_semana_compra") == 7, 'Sab')) \
            .withColumn("ds_mes_compra", when(col("ds_mes_compra") == 1, 'Jan')
            .when(col("ds_mes_compra") == 2, 'Fev')
            .when(col("ds_mes_compra") == 3, 'Mar')
            .when(col("ds_mes_compra") == 4, 'Abr')
            .when(col("ds_mes_compra") == 5, 'Mai')
            .when(col("ds_mes_compra") == 6, 'Jun')
            .when(col("ds_mes_compra") == 7, 'Jul')
            .when(col("ds_mes_compra") == 8, 'Ago')
            .when(col("ds_mes_compra") == 9, 'Set')
            .when(col("ds_mes_compra") == 10, 'Out')
            .when(col("ds_mes_compra") == 11, 'Nov')
            .when(col("ds_mes_compra") == 12, 'Dez'))

In [0]:
df_tickets_dia_semana = df_tratado.groupBy("ds_cluster", "ds_dia_semana_compra").agg(
    sum("total_tickets_quantity_success").alias("qt_total_ticket_dia_semana")
    )

df_compras_mes = df_tratado.groupBy("ds_cluster", "ds_mes_compra").agg(
    sum("gmv_success").alias("gmv_success_total_mes"),
    count("nk_ota_localizer_id").alias("qt_total_compra_mes")
    ).withColumn("vl_medio_compra_mes", round(col("gmv_success_total_mes") / col("qt_total_compra_mes"), 2)) \
    .drop("gmv_success_total_mes")

df_compras_hora = df_tratado.groupBy("ds_cluster", "nu_hora_compra").agg(count("nk_ota_localizer_id").alias("qt_total_compra_hora"))

df_compras_estacao = df_tratado.groupBy("ds_cluster", "ds_estacao_do_ano").agg(count("nk_ota_localizer_id").alias("qt_total_compra_estacao"))

df_compras_dia = df_tratado.groupBy("ds_cluster", "nu_dia_compra").agg(count("nk_ota_localizer_id").alias("qt_total_compra_dia"))

df_compras_rota = df_tratado.groupBy("ds_cluster", "ds_rota_ida").agg(count("nk_ota_localizer_id").alias("qt_total_compra_rota"))

#### Escrita nas tabelas relacionais temporais
---

In [0]:
df_tickets_dia_semana.write.mode("overwrite").saveAsTable(tabela_temporal_ticket_dia_semana)

df_compras_mes.write.mode("overwrite").saveAsTable(tabela_temporal_compras_mes)

df_compras_hora.write.mode("overwrite").saveAsTable(tabela_temporal_compras_hora)

df_compras_estacao.write.mode("overwrite").saveAsTable(tabela_temporal_compras_estacao)

df_compras_dia.write.mode("overwrite").saveAsTable(tabela_temporal_compras_dia)

df_compras_rota.write.mode("overwrite").saveAsTable(tabela_temporal_compras_rota)